In [ ]:
%load_ext autoreload
%autoreload 2

### Description

Generate a conditional output distribution according to some constraints

In [ ]:
import numpy as np
import function_generation
import probability_distributions

In [ ]:
#distribution parameters
input_variables = 4
number_of_states = 5
nudge_size = 0.01

#generate both input and conditional output with Dirichlet weights
distribution_shape = [number_of_states]*input_variables
total_number_of_states = reduce(lambda x,y: x*y, distribution_shape)
input_dist = np.random.dirichlet([1]*total_number_of_states)
input_dist = np.reshape(input_dist, distribution_shape)
cond_shape = [number_of_states]*(input_variables+1)
cond_output = [
    probability_distributions.compute_joint_uniform_random((number_of_states,))
    for i in range(number_of_states**(input_variables))
]
cond_output = np.array(cond_output)
cond_output = np.reshape(cond_output, cond_shape)


In [ ]:
PATH = "/home/derkjan/Documents/academics_UVA/master_thesis/code/"
INPUT_FOLDER = "input_distributions/"
COND_OUTPUT_FOLDER = "conditional_output_distributions/"
FOLDER_FORMAT_INPUT = "{}var_{}states/"
FOLDER_FORMAT_CONDITIONAL = "{}var_{}states/"
FILE_FORMAT_INPUT = "dist_{}.npy"
FILE_FORMAT_COND_OUTPUT = "cond_dist_{}.npy"

DIRICHLET_FOLDER_INPUT = "dirichlet/"
ENTROPY_LOW_FOLDER_INPUT = "entropy_0.5/"
ENTROPY_MEDIUM_FOLDER_INPUT = "entropy_0.75/"

DIRICHLET_FOLDER_COND_OUTPUT = "dirichlet/"


def generate_distributions(path_to_files, file_format, number_of_distributions):
    for i in range(number_of_distributions):
        file_name = path_to_files + file_format.format(i)
        with open(file_name, 'rb') as f:
            yield np.load(f)

def get_input_dist_entropy_75(number_of_dists, number_of_var):
    number_of_states = 5
    path_to_input_files = (
        PATH + INPUT_FOLDER + ENTROPY_MEDIUM_FOLDER_INPUT
        + FOLDER_FORMAT_INPUT.format(number_of_var, number_of_states)
    )
    input_generator = generate_distributions(
        path_to_input_files, FILE_FORMAT_INPUT, 
        number_of_dists
    )
    input_shape = [number_of_states]*number_of_var
    input_dists = []
    for sample in range(number_of_dists):
        input_dist = next(input_generator)
        input_dists.append(np.reshape(input_dist, input_shape))
        
    return input_dists


In [ ]:
number_of_vars = 4
number_of_input_dists = 100
input_dists = input_dists = get_input_dist_entropy_75(
    number_of_input_dists, number_of_vars
)
individual = function_generation.ConditionalOutput(cond_output)
individual.evaluate(input_dists=input_dists, goal_distance=0)
individual.score

In [ ]:
evolutionary_parameters = {
    "number_of_generations": 100,
    "population_size": 10,
    "number_of_children": 20, 
    "generational": True,
    "mutation_size": 0.05,
    "parent_selection_mode": "rank_exponential",
    "number_of_input_distributions": 200
}
number_of_vars = 4
number_of_input_dists = 100
input_dists = input_dists = get_input_dist_entropy_75(
    number_of_input_dists, number_of_vars
)
input_shape = 4 * [5]
number_of_states_output = 5
goal_distance = 1

max_cond_output = function_generation.get_cond_output_with_max_distance(
    input_shape, number_of_states_output, goal_distance, 
    evolutionary_parameters, input_dists
)

In [ ]:

mutated_distribution = function_generation.mutate_conditional_distribution_uniform(
    np.copy(cond_output), 0.01
)

In [ ]:
#local nudge optimization
number_of_generations = 250 
population_size = 10
number_of_children = 30 
generational = True
mutation_size = 0.15
parent_selection_mode = "rank_exponential"
#parent_selection_mode = None
number_of_input_distributions = 200
goal_distance = 0

number_of_states = 5
number_of_input_variables = 4 

#create the initial population
conditional_outputs = function_generation.create_condional_distributions(
    population_size, number_of_states, number_of_input_variables
)

for conditional_output in conditional_outputs:
    conditional_output.evaluate(goal_distance, input_dists)
print("initial distance condtional output {}".format(
    function_generation.sort_individuals(conditional_outputs)[-1].score
))

In [ ]:


#start the optimization process
find_conditional_output = function_generation.FindConditionalOutput(
    conditional_outputs, goal_distance, number_of_generations,
    number_of_children, parent_selection_mode
)
print([i.score for i in find_conditional_output.individuals])
#find_conditional_output.evolve(generational, mutation_size, None,
#                               number_of_input_distributions)
find_conditional_output.evolve(generational, mutation_size, input_dists, None)

print("distance after evolution {}".format(
    function_generation.sort_individuals(find_conditional_output.individuals)[-1].score
))




In [ ]:
find_conditional_output.individuals[-1].cond_output